# 2 pandas中的case_when()新方法

## 2.1 case_when()的使用

In [1]:
import numpy as np
import pandas as pd

# 基础用法
s = pd.Series(np.random.uniform(0, 1, 10))
s

0    0.116089
1    0.142419
2    0.677500
3    0.459483
4    0.176353
5    0.127321
6    0.118835
7    0.333232
8    0.985936
9    0.196752
dtype: float64

In [2]:
s.case_when(
    [
        (s < 0.33, '低水平'),
        ((s >= 0.33) & (s < 0.66), '中水平'),
        (s >= 0.66, '高水平'),
    ]
)

0    低水平
1    低水平
2    高水平
3    中水平
4    低水平
5    低水平
6    低水平
7    中水平
8    高水平
9    低水平
dtype: object

In [3]:
# 函数式条件
s.case_when(
    [
        (lambda x: x < 0.33, '低水平'),
        (lambda x: (x >= 0.33) & (x < 0.66), '中水平'),
        (lambda x: x >= 0.66, '高水平'),
    ]
)

0    低水平
1    低水平
2    高水平
3    中水平
4    低水平
5    低水平
6    低水平
7    中水平
8    高水平
9    低水平
dtype: object

In [4]:
demo_df = pd.read_csv('./二手房数据.csv', encoding='gbk').iloc[:, :-2]
demo_df.head(1)

,市区,小区,户型,朝向,楼层,装修情况,电梯,面积(㎡),价格(万元),年份
0,朝阳,育慧里一区,1室0厅,西,7,精装,有电梯,52,343,2001


In [5]:
(
    demo_df
    # 筛选朝南的有电梯房源
    .query('朝向.str.contains("南") and 电梯 == "有电梯"')
    # 提取合法格式年份并转数值型
    .query('年份.str.fullmatch("\d{4}")')
    .eval('年份=年份.astype("int")')
    # 示例：以年份是否大于中位数来标记是否为老旧房源
    .assign(
        老旧房源=lambda df: df.年份.case_when(
            [
                (lambda s: s <= s.median(), '是'),
                (lambda s: s > s.median(), '否'),
            ]
        )
    )
)

,市区,小区,户型,朝向,楼层,装修情况,电梯,面积(㎡),价格(万元),年份,老旧房源
1,朝阳,大西洋新城A区,2室2厅,南北,10,精装,有电梯,86,835,1999,是
3,朝阳,尚家楼48号院,2室1厅,南北,12,精装,有电梯,75,610,1998,是
5,朝阳,安慧里四区,2室2厅,南西,20,精装,有电梯,71,545,1990,是
6,朝阳,泰华滨河苑,2室1厅,南北,18,精装,有电梯,105,670,2005,是
7,朝阳,北京香颂,3室2厅,南北,17,简装,有电梯,146,1200,2009,否
...,...,...,...,...,...,...,...,...,...,...,...
23659,东城,富贵园一区,3室2厅,南北,9,精装,有电梯,161,1780,2003,是
23665,西城,信德园,3室1厅,西南,23,其他,有电梯,146,960,2002,是
23669,西城,六铺炕二区,3室1厅,西南,17,其他,有电梯,78,1120,1990,是
23671,西城,平原里小区,2室1厅,南北,12,简装,有电梯,59,596,1999,是
